In [1]:



import numpy as np
import os
import json
import glob
Max_tag_number=3

In [2]:
datapath=r'B:\pj\lastfm_subset'

In [3]:
assert os.path.isdir(datapath),'wrong path'

In [4]:
def readjson(path):
    with open(path) as f:
        data=json.load(f)
    return data   

In [5]:
mus_Track_id=[]#存储 trackid
mus_simi_id=[]#只保存最相近的
mus_simi_factor=[]#保存相近系数
mus_tag_Number=[]
mus_tag_Name=[]
mus_tag1=[]
for root,dirs,files in os.walk(datapath):
    
    files=glob.glob(os.path.join(root,'*'+'.json'))
    for f in files:
        f=readjson(f)
        sim=np.array(f['similars'])
        mus_Track_id.append(np.array(f['track_id']))
        tag=np.array(f['tags'])
        if len(sim):
            mus_simi_id.append(sim[0][0])
            mus_simi_factor.append(sim[0][1])
            if len(tag):
                if len(tag[:,0])>=Max_tag_number:
                    mus_tag_Name.append(tag[0:Max_tag_number,0])
                    mus_tag_Number.append(tag[0:Max_tag_number,1])
                    for t in tag[0:2,0]:
                        mus_tag1.append(t)
                    #mus_tag_Name.append(1)
                    
                else:
                    mus_tag_Name.append(tag[:,0])
                    mus_tag_Number.append(tag[:,1])
                    for t in tag[:,0]:
                        mus_tag1.append(t)
                    #mus_tag_Name.append(1)
            else:
                mus_tag_Name.append(0)
                mus_tag_Number.append(0)
                               
        else:
            mus_simi_id.append(0)
            mus_simi_factor.append(0)
            if len(tag):
                if len(tag[:,0])>=Max_tag_number:
                    mus_tag_Name.append(tag[0:Max_tag_number,0])
                    mus_tag_Number.append(tag[0:Max_tag_number,1])
                    for t in tag[0:2,0]:
                        mus_tag1.append(t)
                    #mus_tag_Name.append(1)
                    
                else:
                    mus_tag_Name.append(tag[:,0])
                    mus_tag_Number.append(tag[:,1])
                    for t in tag[:,0]:
                        mus_tag1.append(t)
                    #mus_tag_Name.append(1)
            else:
                mus_tag_Name.append(0)
                mus_tag_Number.append(0)

In [6]:
unique_data = np.unique(mus_tag1)
name=np.array(mus_tag_Name)
number=np.array(mus_tag_Number)
mid=np.zeros([9330,Max_tag_number],dtype=np.float)
for i in range(len(name)):
    c=0
    if number[i]!=0:
        for j in number[i]:
            mid[i][c]=j
            c+=1
        
#cou=[]

midArray=np.zeros([9330,1909],dtype=np.float)
for i in range(len(name)):
    
    if name[i]!=0:
        
        pp=0
        for j in name[i]:
            
            for c in range(1909):
                if j==unique_data[c]:
                    midArray[i,c]=mid[i,pp]
                   
            pp=pp+1
           


c:\Users\GXY\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
c:\Users\GXY\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [7]:
#np.set_printoptions(threshold=np.inf)

In [8]:
#Now we want build a reward function, this function take x,y,w as input.
#x is the tag squence we rececive, 
#y is the result we have receviced from the user (-100,100) 100 means like, -100 means dislike
#w is the weight factor data collected more recently has more power in this function.
#The reward function will take all the informations and compute the best reward result from the function.
#At present we use linear model

In [9]:
from sklearn.linear_model import LinearRegression
def search(tag):
    for i in range(len(mus_Track_id)):
        if mus_Track_id[i]==tag:
            return midArray[i,:]
        else:
            return 0

In [10]:
#WholeData 是剩余所有数据的tag表
#Near_tag 是这些track 最相近的tag表*对应的系数
#near_tag_x is the near tag for x
def Reward(x,y,near_tag_x,z,WholeData,near_tag):
    #对于已经得到结果的track 如果这个track的tag向量是0  那么用他最相近的那个替换
    x=np.array(x)
    y=np.array(y)
    #for i in len(x[:,0]):
              
        #if x[i,:]==0:
            
            #x[i,:]=near_tag_x[i,:]
            
    model=LinearRegression()
    model.fit(x,y)
    y_predict=(model.predict(WholeData)+model.predict(near_tag))*0.5
    return np.argmax(y_predict)
#mus_tag_Name
mus_simi_factor=np.array(mus_simi_factor,dtype=np.float)
#mus_simi_factor
midArray.shape

(9330, 1909)

In [11]:
near_tag=np.zeros([9330,1909],dtype=np.float)
for i in range (len(mus_simi_id)):
    t1=search(mus_simi_id[i])
    
    if t1!=0:
        near_tag[i,:]=t1*mus_simi_factor[i]



In [12]:
#This function is used to add the known tag into know squence
#N is the index of the remove track
def delete1(n,x,y,near_tag_x,yk,wholedata,near_tag):
    recommend=wholedata[n,:]
    near_recommend=near_tag[n,:]
    x.append(recommend)
    y.append(yk)
    near_tag_x.append(near_recommend)
    holedata=np.delete(wholedata,n,axis=0)
    ear_tag=np.delete(near_tag,n,axis=0)
    return x,y,near_tag_x,holedata,ear_tag

In [13]:
import random


In [14]:
ty=np.random.randint(-1,1,(9330,1))

In [15]:
for i in range(len(ty[:,0])):
    if ty[i]==-1:
        ty[i]=-100
    else:
        if ty[i]==1:
            ty[i]=100

near_tag.shape
midArray.shape

(9330, 1909)

In [16]:
error=np.zeros([9330,1])
n=0
e=0
x=[]
y=[]
near_tag_x=[]
for i in range (10):
    if ty[n]<0:
        e=e+1
        error[i]=e
    
    x,y,near_tag_x,midArray,near_tag=delete1(n,x,y,near_tag_x,ty[n],midArray,near_tag)
    
    ty=np.delete(ty,n,axis=0)
    n=Reward(x,y,near_tag_x,x,midArray,near_tag)
    

In [20]:
error[8]

array([5.])